In [1]:
%matplotlib nbagg
import matplotlib.pyplot as plt
import time
import numpy as np

import qcodes as qc
import qcodes.instrument.parameter as parameter

qc.halt_bg()
# qc.set_mp_method('spawn')  # force Windows behavior on mac

# this makes a widget in the corner of the window to show and control
# subprocesses and any output they would print to the terminal
qc.show_subprocess_widget()

<IPython.core.display.Javascript object>

pyqtgraph plotting not supported, try "from qcodes.plots.pyqtgraph import QtPlot" to see the full error
No loop running


In [2]:
import qcodes.instrument_drivers.AlazarTech.ATS9870 as ATSdriver
import qcodes.instrument_drivers.AlazarTech.ATS_acquisition_controllers as ats_contr

In [3]:
ats_inst = ATSdriver.AlazarTech_ATS9870(name='Alazar1', server_name="alazar_server")
contr1 = ats_contr.DFT_AcquisitionController(name='contr1', demodulation_frequency=10e6, alazar=None, server_name="alazar_server")
contr1.set_acquisitionkwargs(#mode='NPT',
                 samples_per_record=1024,
                 records_per_buffer=70,
                 buffers_per_acquisition=1,
                 #channel_selection='AB',
                 #transfer_offset=0,
                 #external_startcapture='ENABLED',
                 #enable_record_headers='DISABLED',
                 #alloc_buffers='DISABLED',
                 #fifo_only_streaming='DISABLED',
                 #interleave_samples='DISABLED',
                 #get_processed_data='DISABLED',
                 allocated_buffers=64,
                 #buffer_timeout=1000
)

In [5]:
server=qc.instrument.server.get_instrument_server_manager("alazar_server")

In [17]:
server._server.

True

In [8]:
alazar=server.instruments[0]

In [11]:
alazar

<qcodes.instrument.remote.RemoteInstrument: Alazar1 at 120910680>

In [ ]:
ats_inst.config(#clock_source='INTERNAL_CLOCK',
                sample_rate=100000000,
                #clock_edge='CLOCK_EDGE_RISING',
                #decimation=0,
                #coupling=['AC','AC'],
                channel_range=[2.,2.],
                #impedance=[50,50],
                #bwlimit=['DISABLED','DISABLED'],
                #trigger_operation='TRIG_ENGINE_OP_J',
                #trigger_engine1='TRIG_ENGINE_J',
                trigger_source1='EXTERNAL',
                #trigger_slope1='TRIG_SLOPE_POSITIVE',
                #trigger_level1=128,
                #trigger_engine2='TRIG_ENGINE_K',
                #trigger_source2='DISABLE',
                #trigger_slope2='TRIG_SLOPE_POSITIVE',
                #trigger_level2=128,
                #external_trigger_coupling='AC',
                #external_trigger_range='ETR_5V',
                #trigger_delay=0,
                #timeout_ticks=0
)

In [ ]:
contr1.get_alazar()==ats_inst

In [ ]:
contr1()

In [ ]:
contr1.snapshot()

In [ ]:
contr1.

In [ ]:
ats_inst.allocated_buffers.get()

In [ ]:
ats_inst.clear_buffers()

In [ ]:
t = []
for n, p in ats_inst.parameters.items():
    t.append([n,p.get()])
    
t

In [ ]:
a = parameter.ManualParameter(name="a")

In [ ]:
data = qc.Loop(a[0:10:0.1], delay=0.1).each(contr1).run()

In [ ]:
data.